It starts by loading environment variables using dotenv, which pulls sensitive values like your API key from a .env file instead of hardcoding them. That’s good operational hygiene. The OPENAI_API_KEY is retrieved from the environment so it can be used securely elsewhere in your project.

The headers dictionary defines a standard browser User-Agent string. Many websites block requests that look like bots, so pretending to be a regular browser increases the chance the request succeeds. It’s a small but practical trick.

The core logic lives in fetch_website_contents(url). Here’s what it does:

First, it sends an HTTP GET request to the provided URL using requests. Then it parses the returned HTML using BeautifulSoup, which turns messy markup into a structured tree you can navigate.

It extracts the page title if it exists. After that, it focuses on the <body> section of the page and removes elements that don’t contribute meaningful readable content — specifically <script>, <style>, <img>, and <input> tags. This prevents JavaScript, CSS, and form elements from polluting the text output.

Next, it extracts the visible text from the cleaned body, separating elements with line breaks for readability. Finally, it combines the title and body text into a single string and truncates the result to 2,000 characters. That truncation is practical — especially if the content is later sent to an AI model with token limits.

In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI 
from bs4 import BeautifulSoup
import requests 
from IPython.display import Markdown, display, update_display
load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')

# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]
    
     


Below is the user prompt to generate the ouput of code

In [2]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
        You are looking at a company called: {company_name}
        Here are the contents of its landing page and other relevant pages;
        use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """
    user_prompt += fetch_website_contents(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt
 

This code initializes an OpenAI client using an API key and sets the base URL to OpenRouter. It then streams a chat completion using the "gpt-4.1-mini" model, providing a system prompt to act as a business brochure expert and a user prompt generated from a company's landing page. The streamed response is displayed in real-time as markdown in the notebook using IPython's display utilities.

In [5]:

client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
)
 
stream = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": 'You are expert in creating documents and pdf for business brochures.'},
        {"role": "user", "content": get_brochure_user_prompt('Soldeir front', 'https://www.soldierfront.com/')},
        ],
    stream=True
)    

response = ""

display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

# print(response.choices[0].message.content)

Soldeir Front Company Brochure

---

**About Soldeir Front**  
Soldeir Front is a dynamic company committed to delivering innovative solutions and exceptional services to its clients. With a focus on quality and customer satisfaction, Soldeir Front stands out in its industry through dedication, expertise, and forward-thinking approaches.

**Our Services**  
- Customized Solutions tailored to meet unique business needs  
- Expert Consultancy providing strategic guidance  
- Advanced Technology Integration to optimize operations  
- Comprehensive Support ensuring seamless client experiences  

**Why Choose Us?**  
- Experienced Team of professionals with deep industry knowledge  
- Client-Centered Approach focusing on partnership and collaboration  
- Proven Track Record of successful project delivery  
- Continuous Innovation driving growth and efficiency  

**Get in Touch**  
Partner with Soldeir Front for your next project and experience excellence in service and results.  
Visit our website or contact us at info@soldeirfront.com for more information.  

---

Soldeir Front – Driving Success, Together.